# Progression analyser

In [1]:
import streamlit as st
from streamlit_jupyter import StreamlitPatcher
StreamlitPatcher().jupyter()

## Import env vars

In [2]:
# from dotenv import load_dotenv
# import os
# load_dotenv()

# USE_HERMES = True # If False, env vars below are not required


# PAT = os.getenv("PAT")
# PAT_ID = os.getenv("PAT_ID")
# HERMES_ENDPOINT = os.getenv("HERMES_ENDPOINT")

## Setup Hermes Auth

In [3]:
# from dotenv import load_dotenv
# import base64
# import requests
# import os

# creds = PAT_ID + ":" + PAT
# headers = {
#     "Authorization": f"Basic {base64.b64encode(creds.encode("utf-8")).decode("utf-8")}"
# }

# url = HERMES_ENDPOINT + f"activities"

# response = requests.get(url, headers=headers)
# if response.status_code == 200:
#     print("PAT valid")
# else: 
#     print("PAT invalid")


## Fetch data activities from HERMES

In [4]:
# import re

# year=2025
# unit="B-DAT-200"
# instance="MAR-2-1"

# url = HERMES_ENDPOINT + f"activities?year={year}&unit={unit}&instance={instance}"

# activity_ids = []

# response = requests.get(url, headers=headers)

# if response.status_code == 200:
#     data = response.json()
#     for activity in data["activities"]:
#         activity_ids.append((activity["id"],  re.sub(r'[^0-9]', '', activity["projectTemplate"]["slug"])))
#     activity_ids.sort(key=lambda x: x[1])
#     print(activity_ids)


## Fetch Data for ids

In [5]:
# url = HERMES_ENDPOINT + f"activities/m/[activity_id]/test_results/details/?"


# for index, activity in enumerate(activity_ids):
#     if index + 1 != len(activity_ids):
#         url += f"activity_id={activity[0]}&"
#     else:
#         url += f"activity_id={activity[0]}"

# response = requests.get(url, headers=headers)
# cleaned_data = []
# print(response.status_code)
# if response.status_code == 200:
#     data = response.json()
#     for id in data:
#         cleaned_data.append(data[id])
    

 

## Make percentages

In [6]:

# for data in cleaned_data:
#     if len(data["results"]) > 0:
#         print(data["results"][0]["coverage"])

## Import data

In [7]:
import pandas as pd
import os
import re

path = "./datasets"
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

def getDayNumber(regex: str, filename: str):
    match = re.search(r"databootcampd(\d+)", filename)
    if match:
        number = match.group(1)
        return number
    else:
        print("No match")

RESULTS = {}
for file in files:
    data = pd.read_csv(f"{path}/{file}", sep=";")
    logins = data.get("login").values
    percentage = data.get("test %").values
    dayResults = []
    dayNumber = f"day{getDayNumber(r'databootcampd(\d+)', file)}"

    for index, login in enumerate(logins):
        dayResults.append((login, percentage[index]))

    RESULTS[dayNumber] = dayResults

st.title("Pool Progression – Epitech")
st.caption("Analyse de la progression des étudiants par jour")

# Pool Progression – Epitech

> Analyse de la progression des étudiants par jour

## Display average score per day

In [8]:
import matplotlib.pyplot as plt
import numpy as np

# Moyenne par jour
days = sorted(RESULTS.keys())
averages = []

for day in days:
    scores = [score for _, score in RESULTS[day]]
    averages.append(np.mean(scores))

fig, ax = plt.subplots()
ax.plot(days, averages, marker="o")
ax.set_title("Moyenne de la classe par jour")
ax.set_xlabel("Jour")
ax.set_ylabel("Moyenne (%)")
ax.set_ylim(0, 100)
fig.autofmt_xdate(rotation=45)
fig.tight_layout()

st.subheader("Progression moyenne de la promotion")
st.pyplot(fig)
plt.close(fig)

### Progression moyenne de la promotion

2026-02-24 12:40:59.642 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 12:40:59.883 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 12:41:07.240 
  command:

    streamlit run /mnt/c/Users/$OCP100-HQO13JTOK7BQ/Documents/pool_progression/venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2026-02-24 12:41:07.240 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 12:41:07.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


## Display average progression per personne per day

In [9]:
import os
import re
import matplotlib.pyplot as plt

OUTPUT_DIR = "plots_students"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def day_sort_key(day):
    m = re.search(r"(\d+)", day)
    return int(m.group(1)) if m else 999

days = sorted(RESULTS.keys(), key=day_sort_key)
day_maps = {day: dict(RESULTS[day]) for day in days}

students = sorted({
    email
    for day in days
    for email, _ in RESULTS[day]
})

st.subheader("Progression individuelle par étudiant")
st.write(f"Génération des graphiques dans `{OUTPUT_DIR}/`...")

for email in students:
    scores = [day_maps[day].get(email, 0) for day in days]

    plt.figure()
    plt.plot(days, scores, marker="o")
    plt.title(f"Progression - {email}")
    plt.xlabel("Jour")
    plt.ylabel("Score (%)")
    plt.ylim(0, 100)
    plt.xticks(rotation=45)
    plt.tight_layout()

    safe_email = re.sub(r"[^a-zA-Z0-9_.-]", "_", email)
    filepath = os.path.join(OUTPUT_DIR, f"{safe_email}.png")

    plt.savefig(filepath, dpi=150)
    plt.close()

print("Images générées dans le dossier :", OUTPUT_DIR)

### Progression individuelle par étudiant

Génération des graphiques dans `plots_students/`...

Images générées dans le dossier : plots_students


Top 3 Hardest days

In [10]:
hardestDays = sorted(enumerate(averages), key=lambda x: x)[:3]

indices, values = zip(*hardestDays)
labels = [f'day{i+1}' for i in indices]

fig, ax = plt.subplots()
ax.bar(labels, values)
ax.set_xlabel("Day")
ax.set_ylabel("Average %")
ax.set_title("Top 3 hardest days (Lowest Averages)")

st.subheader("Top 3 jours les plus difficiles")
st.pyplot(fig)
plt.close(fig)

### Top 3 jours les plus difficiles

2026-02-24 12:41:12.956 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 12:41:13.082 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 12:41:13.082 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 12:41:13.082 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


## DUMP

In [11]:
import json
json.dumps(RESULTS)


'{"day01": [["alexia.regnier@epitech.eu", 100.0], ["alix.ducci@epitech.eu", 42.9], ["aymeric.deroudilhe@epitech.eu", 42.9], ["bastien.lecomte@epitech.eu", 42.9], ["celia.marie-louise@epitech.eu", 42.9], ["elyes.ouahrani@epitech.eu", 57.1], ["ethan.cathebras@epitech.eu", 42.9], ["gautier.oudart@epitech.eu", 57.1], ["idriss.abdellaoui@epitech.eu", 57.1], ["ilies.mammar-tayeb@epitech.eu", 42.9], ["lila.hamri@epitech.eu", 57.1], ["loic.ruyssen@epitech.eu", 71.4], ["lorenzo.philippon@epitech.eu", 100.0], ["louis.savon@epitech.eu", 21.4], ["marc-aurele.homand@epitech.eu", 85.7], ["mathias.rodrigues@epitech.eu", 57.1], ["matteo.castronovo@epitech.eu", 42.9], ["mohamed-amin.belhedi@epitech.eu", 0.0], ["mohamed-yasin.belhedi@epitech.eu", 0.0], ["pierre.untersinger@epitech.eu", 42.9], ["thersan.jean@epitech.eu", 42.9], ["ugo2.cani@epitech.eu", 57.1], ["valentin.barrois@epitech.eu", 42.9], ["yann.rochas@epitech.eu", 57.1]], "day02": [["alexia.regnier@epitech.eu", 100.0], ["alix.ducci@epitech.eu",